<a href="https://colab.research.google.com/github/lucascodinglab/Probabilistic_Random_Forest/blob/main/Pr%C3%A4sentation_Seminar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

In [ ]:
import IPython.display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn.linear_model
import os
import urllib
import IPython.display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn.linear_model
import os
import urllib
from google.colab import data_table
data_table.enable_dataframe_formatter()
from sklearn.preprocessing import LabelEncoder
import math
import warnings
from sklearn.impute import SimpleImputer
from tqdm import tqdm
from numpy.core.numeric import count_nonzero
from sklearn.model_selection import train_test_split
from heapq import nsmallest
from sklearn.metrics import accuracy_score
import random

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/P_Seminar/2_heart_dataset.csv")
df.head()

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,No,16.60,Yes,No,No,3.0,30.0,No,Female,55-59,White,Yes,Yes,Very good,5.0,Yes,No,Yes
1,No,20.34,No,No,Yes,0.0,0.0,No,Female,80 or older,White,No,Yes,Very good,7.0,No,No,No
2,No,26.58,Yes,No,No,20.0,30.0,No,Male,65-69,White,Yes,Yes,Fair,8.0,Yes,No,No
3,No,24.21,No,No,No,0.0,0.0,No,Female,75-79,White,No,No,Good,6.0,No,No,Yes
4,No,23.71,No,No,No,28.0,0.0,Yes,Female,40-44,White,No,Yes,Very good,8.0,No,No,No


In [ ]:
# 30 Percent missing
Thomas = {'HeartDisease':np.nan, 'BMI_cat':2, 'Smoking':0, 'AlcoholDrinking':np.nan, 'Stroke':0,
       'PhysicalHealth_cat':0, 'MentalHealth_cat':0, 'DiffWalking':np.nan, 'Sex':0, 'AgeCategory_cat':0,
       'Race_cat': np.nan, 'Diabetic_cat': 0, 'PhysicalActivity_cat':np.nan, 'GenHealth_cat':0, 'SleepTime_cat':1,
       'Asthma': 0, 'KidneyDisease':np.nan, 'SkinCancer':0}
# 50 Percent missing
Alina = {'HeartDisease':np.nan, 'BMI_cat':1, 'Smoking':np.nan, 'AlcoholDrinking':0, 'Stroke':0,
       'PhysicalHealth_cat':0, 'MentalHealth_cat':np.nan, 'DiffWalking':np.nan, 'Sex':1, 'AgeCategory_cat': np.nan,
       'Race_cat': np.nan, 'Diabetic_cat': np.nan, 'PhysicalActivity_cat':1, 'GenHealth_cat':0, 'SleepTime_cat':2,
       'Asthma': np.nan, 'KidneyDisease':np.nan, 'SkinCancer':0}
# 60 Percent missing
Lucas = {'HeartDisease':np.nan, 'BMI_cat':1, 'Smoking':0, 'AlcoholDrinking':np.nan, 'Stroke':0,
       'PhysicalHealth_cat':0, 'MentalHealth_cat':0, 'DiffWalking':np.nan, 'Sex':0, 'AgeCategory_cat': np.nan,
       'Race_cat': np.nan, 'Diabetic_cat': 0, 'PhysicalActivity_cat':np.nan, 'GenHealth_cat':np.nan, 'SleepTime_cat':np.nan,
       'Asthma': 0, 'KidneyDisease':np.nan, 'SkinCancer':np.nan}

# Import Dataframe

In [ ]:

#shorten the dataset and clear out class inbalance

df['HeartDisease'] = df['HeartDisease'].apply(lambda x: 0 if x.strip()=='No' else 1)

df_grby = df.groupby("HeartDisease")
df0 = df_grby.get_group(0) 
df1 = df_grby.get_group(1)


df0_bt = df0.sample(n = 300, replace = False,random_state = 31) #pick randomly given number of instances from dataframe without replacement
df1_bt = df1.sample(n = 300, replace = False, random_state = 31) 

data_set = pd.concat([df0_bt,df1_bt],ignore_index = True) #join to one dataframe
df = data_set.sample(frac=1,random_state = 31).reset_index(drop=True)

In [ ]:
df["Smoking"] = df["Smoking"].apply(lambda x: 0 if x.strip()=='No' else 1)
df['AlcoholDrinking'] = df['AlcoholDrinking'].apply(lambda x: 0 if x.strip()=='No' else 1)
df["Stroke"] = df["Stroke"].apply(lambda x: 0 if x.strip()=='No' else 1)
df["DiffWalking"] = df["DiffWalking"].apply(lambda x: 0 if x.strip()=='No' else 1)
df["Sex"] = df["Sex"].apply(lambda x: 0 if x.strip()=='Male' else 1)
df["Asthma"] = df["Asthma"].apply(lambda x: 0 if x.strip()=='No' else 1)
df["KidneyDisease"] = df["KidneyDisease"].apply(lambda x: 0 if x.strip()=='No' else 1)
df["SkinCancer"] = df["SkinCancer"].apply(lambda x: 0 if x.strip()=='No' else 1)
df["PhysicalActivity"] = df["PhysicalActivity"].apply(lambda x: 0 if x.strip()=='No' else 1)

one_hot_encoded_data = df.copy()
# Creating a instance of label Encoder.
le = LabelEncoder()

label = le.fit_transform(df["Race"])
one_hot_encoded_data["Race"] = label
one_hot_encoded_data.rename(columns={"Race":"Race_cat"}, inplace = True)

label = le.fit_transform(df["Diabetic"])
one_hot_encoded_data["Diabetic"] = label
one_hot_encoded_data.rename(columns={"Diabetic":"Diabetic_cat"}, inplace = True)

label = le.fit_transform(df["GenHealth"])
one_hot_encoded_data["GenHealth"] = label
one_hot_encoded_data.rename(columns={"GenHealth":"GenHealth_cat"}, inplace = True)

# Using .fit_transform function to fit label
# encoder and return encoded label
label = le.fit_transform(df['AgeCategory'])
one_hot_encoded_data["AgeCategory"] = label
one_hot_encoded_data["AgeCategory"] = pd.cut(one_hot_encoded_data['AgeCategory'], bins=[0, 3, 5, 8, 10, 13])
one_hot_encoded_data.rename(columns={"AgeCategory":"AgeCategory_cat"}, inplace = True)
label = le.fit_transform(one_hot_encoded_data['AgeCategory_cat'])
one_hot_encoded_data["AgeCategory_cat"]=label

#Split BMI numerical Data into categorial Data by the official BMI Categories
one_hot_encoded_data["BMI"] = pd.cut(one_hot_encoded_data['BMI'], bins=[0, 20, 23, 27, 32, 36, 99])
one_hot_encoded_data.rename(columns={"BMI":"BMI_cat"}, inplace = True)
label = le.fit_transform(one_hot_encoded_data['BMI_cat'])
one_hot_encoded_data["BMI_cat"]=label

#Split PhysicalHealth numerical Data into categorial Data by the official PhysicalHealth Categories
one_hot_encoded_data["PhysicalHealth"] = pd.cut(one_hot_encoded_data['PhysicalHealth'], bins=[0,5,10,31])
one_hot_encoded_data.rename(columns={"PhysicalHealth":"PhysicalHealth_cat"}, inplace = True)
label = le.fit_transform(one_hot_encoded_data['PhysicalHealth_cat'])
one_hot_encoded_data["PhysicalHealth_cat"]=label

#Split MentalHealth numerical Data into categorial Data by the official MentalHealth Categories
one_hot_encoded_data["MentalHealth"] = pd.cut(one_hot_encoded_data['MentalHealth'], bins=[0,5,10,31])
one_hot_encoded_data.rename(columns={"MentalHealth":"MentalHealth_cat"}, inplace = True)
label = le.fit_transform(one_hot_encoded_data['MentalHealth_cat'])
one_hot_encoded_data["MentalHealth_cat"]=label

#Split SleepTime numerical Data into categorial Data by the official SleepTime Categories
one_hot_encoded_data["SleepTime"] = pd.cut(one_hot_encoded_data['SleepTime'], bins=[0,5,7,9,10,25])
one_hot_encoded_data.rename(columns={"SleepTime":"SleepTime_cat"}, inplace = True)
label = le.fit_transform(one_hot_encoded_data['SleepTime_cat'])
one_hot_encoded_data["SleepTime_cat"]=label



In [ ]:


one_hot_encoded_data = one_hot_encoded_data.append(Thomas, ignore_index = True)
one_hot_encoded_data = one_hot_encoded_data.append(Alina, ignore_index = True)
one_hot_encoded_data = one_hot_encoded_data.append(Lucas, ignore_index = True)

# 0. Probabilities - KNN Algorithm

In [ ]:
#randomly delete 20 percent of data and replace it with NaN-values
#delete Diabetes_012 column
data_delete = one_hot_encoded_data.drop(columns='HeartDisease')
# get dimensions of data_delete
n_rows, n_cols = len(data_delete.index), len(data_delete.columns)       
volume = n_rows * n_cols         
volume_nan = int(volume * 0.5)      

# create random index locations for NaNs
index = np.random.seed(32)
index = np.random.randint(volume, size=volume_nan)
row_index = index % n_rows
col_index = (index / n_rows).astype(int)


# assign NaN to each of the indices in data_delete
for ri, ci in zip(row_index, col_index):
  data_delete.iloc[ri, ci] = np.nan

data_delete.head(10)  

,BMI_cat,Smoking,AlcoholDrinking,Stroke,PhysicalHealth_cat,MentalHealth_cat,DiffWalking,Sex,AgeCategory_cat,Race_cat,Diabetic_cat,PhysicalActivity,GenHealth_cat,SleepTime_cat,Asthma,KidneyDisease,SkinCancer,PhysicalActivity_cat
0,NaN,1.0,0.0,NaN,NaN,3.0,NaN,NaN,2.0,NaN,0.0,1.0,4.0,NaN,NaN,0.0,NaN,NaN
1,NaN,1.0,NaN,NaN,3.0,3.0,0.0,NaN,2.0,NaN,0.0,1.0,2.0,1.0,0.0,NaN,0.0,NaN
2,NaN,NaN,NaN,0.0,NaN,3.0,0.0,NaN,NaN,NaN,NaN,1.0,NaN,2.0,NaN,0.0,NaN,NaN
3,NaN,1.0,0.0,0.0,2.0,NaN,1.0,NaN,NaN,2.0,NaN,NaN,1.0,2.0,0.0,NaN,NaN,NaN
4,NaN,NaN,0.0,NaN,1.0,2.0,NaN,NaN,NaN,NaN,0.0,NaN,2.0,0.0,NaN,0.0,NaN,NaN
5,4.0,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,5.0,1.0,1.0,NaN,NaN,NaN,0.0,NaN,NaN
6,NaN,1.0,NaN,0.0,NaN,3.0,NaN,1.0,NaN,5.0,NaN,1.0,1.0,2.0,0.0,1.0,0.0,NaN
7,NaN,1.0,0.0,NaN,NaN,3.0,0.0,0.0,3.0,5.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN
8,1.0,0.0,0.0,0.0,NaN,3.0,0.0,1.0,NaN,5.0,NaN,1.0,NaN,1.0,0.0,0.0,0.0,NaN
9,5.0,NaN,1.0,0.0,NaN,2.0,1.0,0.0,3.0,5.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN


In [ ]:
# Method 1 : Filling the NaN-values with k-nearest neighbor algorithm
# For each NaN-value: calculating the k-nearest neighbors + calculating based on that the probability of each occurence of an attribute
#from pandas.core import indexing

# replace Nan-values with mean
df_mean = one_hot_encoded_data.copy()
df_mean["HeartDisease"] = one_hot_encoded_data["HeartDisease"]
mean_imputer = SimpleImputer(strategy='mean')
def pre_nan():  
  for i in range(0,17):
    df_mean.iloc[:,[i]] = mean_imputer.fit_transform(df_mean.iloc[:,[i]].values.reshape(-1,1))
pre_nan()


from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
transformed = ohe.fit_transform(df_mean)
df_mean = transformed.toarray()

array_datamean = df_mean
#array_datamean = df_mean.to_numpy()


# functions for calculating the k nearest neighbor
import random
# calculates the euclidean distance between two arrays
def euclidean_distance(array1, array2):
  distance=0.0
  for i in range(len(array1)):
      distance = distance + (array1[i]- array2[i])**2
      result = math.sqrt(distance)
  return result

# get nearest neighbors (distances = row values, distance, row index / neighbors = row index of knn)
def get_neighbors(train, test_row, k):
  #train = train.sample(n=1500,replace=False) #Lucas hinzugefügt
  distances = list() 
  for i, train_row in enumerate(train):
    dist = euclidean_distance(test_row, train_row)
    distances.append((train_row, dist,i))
  distances.sort(key=lambda tup: tup[1])  #sorts the distances in ascending order
  neighbors = list()
  for i in range(k+1):                          
    neighbors.append(distances[i][2])
  return neighbors


#counts the occurences of each class in the knn for binary attributes
def count_occbin (listinput, k, column):
  new_valar = []
  new_values = []
  cluster_0 = 0
  cluster_1 = 0
  count = 0
  for x in listinput: 
    val = data_delete.iloc[x][column]
    if (val == 1.0):
      cluster_1 = cluster_1 + 1.0
    elif (val == 0.0):
      cluster_0 = cluster_0 + 1.0
    elif math.isnan(val):             #ignore nan values in calculation
      count = count + 1
      continue
  division = k-count
  if division == 0:
    division = 0.0000000000000000000001
  new_valar = [(cluster_0/division),(cluster_1/division)] 
  new_values.append(new_valar)
  new_values = np.array(new_values)
  return new_values

#counts the occurences of each class in the knn for categorial attributes
def count_occcat (listinput,k,column): 
  new_valar = []
  new_values = []
  cluster_0 = 0
  cluster_1 = 0
  cluster_2 = 0
  cluster_3 = 0
  cluster_4 = 0
  cluster_5 = 0
  count = 0
  for x in listinput: 
    val = data_delete.iloc[x][column]
    if (val == 0.0):
      cluster_0 = cluster_0 + 1.0
    elif (val == 1.0):
      cluster_1 = cluster_1 + 1.0
    elif (val == 2.0):
      cluster_2 = cluster_2 + 1.0
    elif (val == 3.0):
      cluster_3 = cluster_3 + 1.0
    elif (val == 4.0):
      cluster_4 = cluster_4 + 1.0
    elif (val == 5.0):
      cluster_5 = cluster_5 + 1.0
    elif math.isnan(val):             #ignore nan values in calculation
      count = count + 1
      continue
  division = k-count
  if division == 0:
    division = 0.0000000000000000000001
  new_valar = [(cluster_0/division),(cluster_1/division),(cluster_2/division), (cluster_3/division), (cluster_4/division), (cluster_5/division)]
  new_values.append(new_valar)
  new_values = np.array(new_values)
  return new_values


  # gets k nearest neighbors from NaN values and creates new probabilities
from tqdm import tqdm
# fill new table for binary attributes
def filltablebin (x):
  newcolumn1 =[]
  newcolumn0 = []
  for i, row in tqdm(data_delete.iterrows()): 
    if math.isnan(row[x]) == False:
      if row[x] == 1.0:
        newcolumn1.append(1.0)
        newcolumn0.append(0.0)
      elif row[x] == 0.0:
           newcolumn1.append(0.0)
           newcolumn0.append(1.0)
    elif math.isnan(row[x]):                                         
         neighbors = get_neighbors(array_datamean, array_datamean[i], 15)  
         neighbors.remove(i)   #removes own row index out of neighbors (as it is always distance 0)
         prob_nan = count_occbin(neighbors,15,x) #here
         one = prob_nan[0,1]
         zero = prob_nan[0,0]
         newcolumn1.append(one)
         newcolumn0.append(zero)
  return newcolumn0, newcolumn1

# fill new table for categorial attributes
def filltablecat (x):
  newcolumn0 = []
  newcolumn1 =[]
  newcolumn2 = []
  newcolumn3 = []
  newcolumn4 = []
  newcolumn5 = []
  arr = []
  for i, row in tqdm(data_delete.iterrows()): 
    if math.isnan(row[x]) == False:
      if row[x] == 0.0:
        newcolumn1.append(0.0)
        newcolumn0.append(1.0)
        newcolumn2.append(0.0)
        newcolumn3.append(0.0)
        newcolumn4.append(0.0)
        newcolumn5.append(0.0)
      elif row[x] == 1.0:
        newcolumn1.append(1.0)
        newcolumn0.append(0.0)
        newcolumn2.append(0.0)
        newcolumn3.append(0.0)
        newcolumn4.append(0.0)
        newcolumn5.append(0.0)
      elif row[x] == 2.0:
        newcolumn1.append(0.0)
        newcolumn0.append(0.0)
        newcolumn2.append(1.0)
        newcolumn3.append(0.0)
        newcolumn4.append(0.0)
        newcolumn5.append(0.0)
      elif row[x] == 3.0:
        newcolumn1.append(0.0)
        newcolumn0.append(0.0)
        newcolumn2.append(0.0)
        newcolumn3.append(1.0)
        newcolumn4.append(0.0)
        newcolumn5.append(0.0)
      elif row[x] == 4.0:
        newcolumn1.append(0.0)
        newcolumn0.append(0.0)
        newcolumn2.append(0.0)
        newcolumn3.append(0.0)
        newcolumn4.append(1.0)
        newcolumn5.append(0.0)
      elif row[x] == 5.0:
        newcolumn1.append(0.0)
        newcolumn0.append(0.0)
        newcolumn2.append(0.0)
        newcolumn3.append(0.0)
        newcolumn4.append(0.0)
        newcolumn5.append(1.0)
    elif math.isnan(row[x]):                                     
         neighbors = get_neighbors(array_datamean, array_datamean[i], 15)   
         neighbors.remove(i)   #removes own row index out of neighbors (as it is always distance 0)
         prob_nan = count_occcat(neighbors,15,x)
         zero = prob_nan[0,0]
         one = prob_nan[0,1]
         two = prob_nan[0,2]
         three = prob_nan[0,3]
         four = prob_nan[0,4]
         five = prob_nan[0,5]
         newcolumn0.append(zero)
         newcolumn1.append(one)
         newcolumn2.append(two)
         newcolumn3.append(three)
         newcolumn4.append(four)
         newcolumn5.append(five)
         arr = newcolumn0, newcolumn1, newcolumn2, newcolumn3, newcolumn4, newcolumn5
  return arr



  

In [ ]:
#fill new table with probabilities
columns = ['HeartDisease', 'Smoking_No', 'Smoking_Yes', 'AlcoholDrinking_No',
       'AlcoholDrinking_Yes', 'DiffWalking_No', 'DiffWalking_Yes',
       'Sex_Female', 'Sex_Male', 'Race_American Indian/Alaskan Native',
       'Race_Asian', 'Race_Black', 'Race_Hispanic', 'Race_Other', 'Race_White',
       'Diabetic_No', 'Diabetic_No, borderline diabetes', 'Diabetic_Yes',
       'Diabetic_Yes (during pregnancy)', 'PhysicalActivity_No',
       'PhysicalActivity_Yes', 'GenHealth_Excellent', 'GenHealth_Fair',
       'GenHealth_Good', 'GenHealth_Poor', 'GenHealth_Very good', 'Asthma_No',
       'Asthma_Yes', 'KidneyDisease_No', 'KidneyDisease_Yes', 'SkinCancer_No',
       'SkinCancer_Yes', 'Stroke_No', 'Stroke_Yes', 'BMI_cat_0', 'BMI_cat_1',
       'BMI_cat_2', 'BMI_cat_3', 'BMI_cat_4', 'BMI_cat_5',
       'PhysicalHealth_cat_0', 'PhysicalHealth_cat_1', 'PhysicalHealth_cat_2',
       'PhysicalHealth_cat_3', 'MentalHealth_cat_0', 'MentalHealth_cat_1',
       'MentalHealth_cat_2', 'MentalHealth_cat_3', 'AgeCategory_cat_0',
       'AgeCategory_cat_1', 'AgeCategory_cat_2', 'AgeCategory_cat_3',
       'AgeCategory_cat_4', 'AgeCategory_cat_5', 'SleepTime_cat_0',
       'SleepTime_cat_1', 'SleepTime_cat_2', 'SleepTime_cat_3',
       'SleepTime_cat_4']
df_splitknn = pd.DataFrame(columns=columns)
newcol_smoking = list()
newcol_alcoholdrinking = list()
newcol_diffwalking = list()
newcol_sex = list()
newcol_race = list()
newcol_diabetic = list()
newcol_physicalactivity = list()
newcol_genhealth = list()
newcol_asthma = list()
newcol_kidneydisease = list()
newcol_skincancer = list()
newcol_stroke = list()
newcol_bmi = list()
newcol_physicalhealth = list()
newcol_mentalhealth = list()
newcol_age = list()
newcol_sleeptime = list()

df_splitknn['HeartDisease'] = one_hot_encoded_data['HeartDisease']

newcol_smoking = filltablebin('Smoking')
df_splitknn["Smoking_No"]= newcol_smoking[0]
df_splitknn['Smoking_Yes'] = newcol_smoking[1]

newcol_alcoholdrinking = filltablebin('AlcoholDrinking')
df_splitknn['AlcoholDrinking_No'] = newcol_alcoholdrinking[0]
df_splitknn['AlcoholDrinking_Yes'] =newcol_alcoholdrinking[1]

newcol_diffwalking = filltablebin('DiffWalking')
df_splitknn['DiffWalking_No'] = newcol_diffwalking[0]
df_splitknn['DiffWalking_Yes'] =newcol_diffwalking[1]

newcol_sex = filltablebin('Sex')
df_splitknn['Sex_Male'] = newcol_sex[0]
df_splitknn['Sex_Female'] =newcol_sex[1]

newcol_race = filltablecat('Race_cat')
df_splitknn['Race_American Indian/Alaskan Native'] = newcol_race[0]
df_splitknn['Race_Asian'] =newcol_race[1]
df_splitknn['Race_Black'] =newcol_race[2]
df_splitknn['Race_Hispanic'] =newcol_race[3]
df_splitknn['Race_Other'] =newcol_race[4]
df_splitknn['Race_White'] =newcol_race[5]


newcol_diabetic = filltablecat('Diabetic_cat')
df_splitknn['Diabetic_No'] = newcol_diabetic[0]
df_splitknn['Diabetic_No, borderline diabetes'] =newcol_diabetic[1]
df_splitknn['Diabetic_Yes'] =newcol_diabetic[2]
df_splitknn['Diabetic_Yes (during pregnancy)'] =newcol_diabetic[3]


newcol_physicalactivity = filltablebin('PhysicalActivity')
df_splitknn['PhysicalActivity_No'] = newcol_physicalactivity[0]
df_splitknn['PhysicalActivity_Yes'] =newcol_physicalactivity[1]

newcol_genhealth = filltablecat('GenHealth_cat')
df_splitknn['GenHealth_Excellent'] = newcol_genhealth[0]
df_splitknn['GenHealth_Fair'] =newcol_genhealth[1]
df_splitknn['GenHealth_Good'] =newcol_genhealth[2]
df_splitknn['GenHealth_Poor'] =newcol_genhealth[3]
df_splitknn['GenHealth_Very good'] =newcol_genhealth[4]

newcol_asthma = filltablebin('Asthma')
df_splitknn['Asthma_No'] = newcol_asthma[0]
df_splitknn['Asthma_Yes'] =newcol_asthma[1]

newcol_kidneydisease = filltablebin('KidneyDisease')
df_splitknn['KidneyDisease_No'] = newcol_kidneydisease[0]
df_splitknn['KidneyDisease_Yes'] =newcol_kidneydisease[1]

newcol_skincancer = filltablebin('SkinCancer')
df_splitknn['SkinCancer_No'] = newcol_skincancer[0]
df_splitknn['SkinCancer_Yes'] =newcol_skincancer[1]

newcol_stroke = filltablebin('Stroke')
df_splitknn['Stroke_No'] = newcol_stroke[0]
df_splitknn['Stroke_Yes'] =newcol_stroke[1]

newcol_bmi = filltablecat('BMI_cat')
df_splitknn['BMI_cat_0'] = newcol_bmi[0]
df_splitknn['BMI_cat_1'] =newcol_bmi[1]
df_splitknn['BMI_cat_2'] =newcol_bmi[2]
df_splitknn['BMI_cat_3'] =newcol_bmi[3]
df_splitknn['BMI_cat_4'] =newcol_bmi[4]
df_splitknn['BMI_cat_5'] =newcol_bmi[5]

newcol_physicalhealth = filltablecat('PhysicalHealth_cat')
df_splitknn['PhysicalHealth_cat_0'] = newcol_physicalhealth[0]
df_splitknn['PhysicalHealth_cat_1'] =newcol_physicalhealth[1]
df_splitknn['PhysicalHealth_cat_2'] =newcol_physicalhealth[2]
df_splitknn['PhysicalHealth_cat_3'] =newcol_physicalhealth[3]

newcol_mentalhealth = filltablecat('MentalHealth_cat')
df_splitknn['MentalHealth_cat_0'] = newcol_mentalhealth[0]
df_splitknn['MentalHealth_cat_1'] =newcol_mentalhealth[1]
df_splitknn['MentalHealth_cat_2'] =newcol_mentalhealth[2]
df_splitknn['MentalHealth_cat_3'] =newcol_mentalhealth[3]

newcol_age = filltablecat('AgeCategory_cat')
df_splitknn['AgeCategory_cat_0'] = newcol_age[0]
df_splitknn['AgeCategory_cat_1'] =newcol_age[1]
df_splitknn['AgeCategory_cat_2'] =newcol_age[2]
df_splitknn['AgeCategory_cat_3'] =newcol_age[3]
df_splitknn['AgeCategory_cat_4'] =newcol_age[4]
df_splitknn['AgeCategory_cat_5'] =newcol_age[5]

newcol_sleeptime = filltablecat('SleepTime_cat')
df_splitknn['SleepTime_cat_0'] = newcol_sleeptime[0]
df_splitknn['SleepTime_cat_1'] =newcol_sleeptime[1]
df_splitknn['SleepTime_cat_2'] =newcol_sleeptime[2]
df_splitknn['SleepTime_cat_3'] =newcol_sleeptime[3]
df_splitknn['SleepTime_cat_4'] =newcol_sleeptime[4]


603it [00:11, 51.37it/s]
603it [00:10, 57.19it/s]
603it [00:13, 44.21it/s]
603it [00:10, 57.04it/s]
603it [00:09, 61.43it/s]
603it [00:11, 54.57it/s]
603it [00:10, 56.43it/s]
603it [00:08, 72.62it/s]
603it [00:10, 55.85it/s]
603it [00:10, 58.14it/s]
603it [00:07, 77.54it/s]
603it [00:12, 48.58it/s]
603it [00:11, 51.41it/s]
603it [00:10, 59.76it/s]
603it [00:08, 68.45it/s]
603it [00:10, 57.65it/s]
603it [00:10, 55.97it/s]


## 2. Entropy calculation

In [ ]:
column_dict = {"Smoking":["Smoking_No","Smoking_Yes"],
               "AlcoholDrinking": ["AlcoholDrinking_No", "AlcoholDrinking_Yes"],
               "DiffWalking": ["DiffWalking_No","DiffWalking_Yes"],
               "Sex":["Sex_Female","Sex_Male"],
               "Race":["Race_American Indian/Alaskan Native","Race_Asian","Race_Black","Race_Hispanic","Race_Other","Race_White"],
               "Diabetic":["Diabetic_No","Diabetic_No, borderline diabetes","Diabetic_Yes","Diabetic_Yes (during pregnancy)"],
               "PhysicalActivity":["PhysicalActivity_No","PhysicalActivity_Yes"],
               "GenHealth":["GenHealth_Excellent","GenHealth_Fair","GenHealth_Good","GenHealth_Poor","GenHealth_Very good"],
               "Asthma":["Asthma_No","Asthma_Yes"],
               "KidneyDisease":["KidneyDisease_No","KidneyDisease_Yes"],
               "SkinCancer":["SkinCancer_No","SkinCancer_Yes"],
               "Stroke":["Stroke_No","Stroke_Yes"],
               "BMI_cat":["BMI_cat_0","BMI_cat_1","BMI_cat_2","BMI_cat_3","BMI_cat_4","BMI_cat_5"],
               "PhysicalHealth_cat":["PhysicalHealth_cat_0","PhysicalHealth_cat_1","PhysicalHealth_cat_2","PhysicalHealth_cat_3"],
               "MentalHealth_cat":["MentalHealth_cat_0","MentalHealth_cat_1","MentalHealth_cat_2","MentalHealth_cat_3"],
               "AgeCategory_cat":["AgeCategory_cat_0","AgeCategory_cat_1","AgeCategory_cat_2","AgeCategory_cat_3","AgeCategory_cat_4","AgeCategory_cat_5"],
               "SleepTime_cat":["SleepTime_cat_0","SleepTime_cat_1","SleepTime_cat_2","SleepTime_cat_3","SleepTime_cat_4"]}

In [ ]:
def entropy(x_col,df,y_col): #calculates the entropy of an occurrence --> First part of evaluation ; y_col: Name of the target variable
  #filter dataframe by type of Diabetes for sum calculation in entropy function
  warnings.filterwarnings("ignore")
  df_grby=df.groupby(y_col)

  a = df[x_col].to_numpy()
  sum_012 = np.sum(a)  #calculate sum of y for hole dataframe

  try:
    df0=df_grby.get_group(0)
    b = df0[x_col].to_numpy()
    sum_0 = np.sum(b)    #calculate sum of filtered dataframe by just diabetes = 0
    var_0=(sum_0/sum_012)  #calculate terms for entropy
    term_0 = var_0*np.log2(var_0)  #calculate term for entropy
    
  except:             #if the grouped by dataframe is empty set var_0 to 0, because the log2(0) raises an error
    term_0 = 0

  try:
    df1=df_grby.get_group(1)
    c = df1[x_col].to_numpy()
    sum_1 = np.sum(c)    #...
    var_1=(sum_1/sum_012)
    term_1 = var_1*np.log2(var_1)
    
  except:
    term_1 = 0 

  try:
    df2=df_grby.get_group(2)   
    d = df2[x_col].to_numpy()
    sum_2 = np.sum(d)
    var_2=(sum_2/sum_012)
    term_2 = var_2*np.log2(var_2)
    

  except:
    term_2 = 0

  if math.isnan(term_0):
    term_0 = 0 
  if math.isnan(term_1):
    term_1 = 0 
  if math.isnan(term_2):
    term_2 = 0 
  #calculate entropy for every occurance
  #entropy = -var_0*np.log2(var_0)-var_1*np.log2(var_1)-var_2*np.log2(var_2)
  entropy = (-term_0 - term_1 - term_2)


  return(sum_012,entropy)

# 3. Update new dataframe after split by multiplication of the probabilities

In [ ]:
def update_df_by_split(split_name,c_dict,df,y_col): #calculate new dataframe for every split attribute - get as input the prior df and the split column as string
    
  #delete all splits to the coresponding attribute from the new dataframe
  key_column=""
  for key in c_dict:
    for x_col in c_dict[key]:
      if x_col == split_name:
        key_column = key 


  value_ary = c_dict[key_column]
  #print(value_ary)
  #print(key_column)

  
  
  y=df[y_col]
  df1=df.drop(y_col,axis=1)
  split_ary = df[split_name]
  df_new = df1 * split_ary[:,np.newaxis]
  df_new[y_col]=y

  for i in value_ary:
    df_new.drop(i,axis=1,inplace=True)
  

  


  column_dict=c_dict.copy()
  column_dict.pop(key_column, None)  
  
  
  #value_ary = column_dict[key_column]
  #print(value_ary)

  return df_new.loc[(df_new.drop(y_col, axis=1)!=0).any(axis=1)],column_dict
    


# 4. Information-gain for every attribute and split

In [ ]:
def info_min_attribute(c_dict,df,y_col):  #sum of the weighted entropies of the individual characteristics of an attribute
  info_min=0
  key_min =""
  for key in c_dict:
    info_value=0
    sum_valcounts = len(df)
    for x_col in c_dict[key]: #iterate through every column key (Smoking, DiffWalking, BMI_cat...)
      #print(x_col)
      sum_012,ent=entropy(x_col,df,y_col)
      #print(sum_012)
      #print(ent)
      info_value+=((sum_012/sum_valcounts)*ent)
    #print("For the ",key," the information value is: ",info_value)
    if info_min == 0:
      info_min = info_value
      key_min = key
    elif info_value < info_min:
      info_min = info_value
      key_min = key


  return (key_min,info_min)

In [ ]:
def info_gain(ent_prior,ent_current): #calculate the infogain for every step
  return ent_prior-ent_current

In [ ]:
def info_min_split(info_min_att,c_dict,df,y_col): 
  #takes as input the name of the attribute with the minimal entropy, calcualte the entropy of every split attribute of the given Attribute with the smallest entropy
  #result is a dataframe with the information status of every split
  split_ary = []
  for x_col in c_dict[info_min_att]:
    sum_012,ent=entropy(x_col,df,y_col)
    new_df, new_dict = update_df_by_split(x_col,c_dict,df,y_col)


    split_ary.append([x_col,new_df,new_dict])
    


  
  return split_ary


In [ ]:
def split_prediction (df,y_col): #calculate the class probabilities for each split

  df_grby=df.groupby(y_col)
  try:
    df0 = df_grby.get_group(0)
    y_0 = len(df0) / len(df)
  except:
    y_0 = 0
  try:
    df1 = df_grby.get_group(1)
    y_1 = len(df1) / len(df)
  except:
    y_1 = 0
  try:
    df2 = df_grby.get_group(2)
    y_2 = len(df2) / len(df)
  except:
    y_2 = 0
  prediction = [float(y_0), float(y_1), float(y_2)]
  return prediction

# 5. Train the probabilistic decision Tree

In [ ]:
def train_tree(c_dict,df_train,min_info_gain,max_depth,y_col): #train the tree by given df and dictionary

  tree_ary = []
  min_att0 = info_min_attribute(c_dict,df_train,y_col)  #returns Attribute with smales entropy, as well as the coresponding entropy of the attribute
  split0 = info_min_split(min_att0[0],c_dict,df_train,y_col)  #returns occurences of the attribute with the samlest entropy incl. new_df and new dict
  counter_lvl0=0
  depth_counter = 0
  for a in tqdm(split0):  #[GenHlth=0,GenHlth=1....]
    split_name0 = a[0]  #GenHlt=0
    new_df0 = a[1]
    new_dict0 = a[2]
    counter_lvl1 = 0
    stop_counter1 = 0
    

    min_att1 = info_min_attribute(new_dict0,new_df0,y_col)  #HighBP
    split1 = info_min_split(min_att1[0],new_dict0,new_df0,y_col)  #[HighBP=1, HighBP=0]
     
    tree_ary.append([split_name0, []])
      
   
    for b in split1:
      split_name1 = b[0] #HighBP=1
      new_df1 = b[1]
      new_dict1 = b[2]
      counter_lvl2 = 0
      stop_counter2 = 0
      depth_counter = 2 
    
      min_att2 = info_min_attribute(new_dict1,new_df1,y_col)
      split2 = info_min_split(min_att2[0],new_dict1,new_df1,y_col) 
      info_gain_1_2 = min_att0[1] - min_att1[1]

  
      if info_gain_1_2 >= min_info_gain and depth_counter<= max_depth:
        tree_ary[counter_lvl0][1].append([split_name1, []])
         
        
      elif stop_counter1 == 0:
        tree_ary[counter_lvl0][1].append([split_prediction(new_df0,y_col)]) 
        stop_counter1+=1 
        continue
      else:
        continue
        
      for c in split2:
        split_name2 = c[0] 
        new_df2 = c[1]
        new_dict2 = c[2] 
        counter_lvl3=0
        stop_counter3 = 0
        depth_counter = 3 

        min_att3 = info_min_attribute(new_dict2,new_df2,y_col)
        split3 = info_min_split(min_att3[0],new_dict2,new_df2,y_col)
        info_gain_2_3 = min_att1[1] - min_att2[1]
        
        if info_gain_2_3 >= min_info_gain and depth_counter<= max_depth:
          tree_ary[counter_lvl0][1][counter_lvl1][1].append([split_name2, []])
          
        
        elif stop_counter2 == 0:
          tree_ary[counter_lvl0][1][counter_lvl1][1].append([split_prediction(new_df1,y_col)]) 
          stop_counter2+=1 
          continue
        else:
          continue       
         
        for d in split3:
          split_name3 = d[0] 
          new_df3= d[1]
          new_dict3 = d[2]
          counter_lvl4 = 0
          stop_counter4 = 0
          depth_counter = 4 

          min_att4 = info_min_attribute(new_dict3,new_df3,y_col)
          split4 = info_min_split(min_att4[0],new_dict3,new_df3,y_col)
          info_gain_3_4 = min_att2[1] - min_att3[1]
       
          if info_gain_3_4 >= min_info_gain and depth_counter<= max_depth:
            tree_ary[counter_lvl0][1][counter_lvl1][1][counter_lvl2][1].append([split_name3, []])
            
        
          elif stop_counter3 == 0:
            tree_ary[counter_lvl0][1][counter_lvl1][1][counter_lvl2][1].append([split_prediction(new_df2,y_col)])
            stop_counter3+=1 
            continue 
          else:
            continue

          for e in split4:
            split_name4 = e[0] 
            new_df4= e[1]
            new_dict4 = e[2]
            counter_lvl5 = 0
            stop_counter5 = 0
            depth_counter = 5 

            min_att5 = info_min_attribute(new_dict4,new_df4,y_col)
            split5 = info_min_split(min_att5[0],new_dict4,new_df4,y_col)
            info_gain_4_5 = min_att3[1] - min_att4[1]
         
            if info_gain_4_5 >= min_info_gain and depth_counter<= max_depth:
              tree_ary[counter_lvl0][1][counter_lvl1][1][counter_lvl2][1][counter_lvl3][1].append([split_name4, []])
            
        
            elif stop_counter4 == 0:
              tree_ary[counter_lvl0][1][counter_lvl1][1][counter_lvl2][1][counter_lvl3][1].append([split_prediction(new_df3,y_col)])
              stop_counter4+=1 
              continue 
            else:
              continue

            for f in split5:
              split_name5 = f[0] 
              new_df5= f[1]
              new_dict5 = f[2]
              counter_lvl6 = 0
              stop_counter6 = 0
              depth_counter = 6 

              min_att6 = info_min_attribute(new_dict5,new_df5,y_col)
              split6 = info_min_split(min_att6[0],new_dict5,new_df5,y_col)
              info_gain_5_6 = min_att4[1] - min_att5[1]
              
              if info_gain_5_6 >= min_info_gain and depth_counter<= max_depth:
                tree_ary[counter_lvl0][1][counter_lvl1][1][counter_lvl2][1][counter_lvl3][1][counter_lvl4][1].append([split_name5, []])
            
        
              elif stop_counter5 == 0:
                tree_ary[counter_lvl0][1][counter_lvl1][1][counter_lvl2][1][counter_lvl3][1][counter_lvl4][1].append([split_prediction(new_df4,y_col)])
                stop_counter5+=1 
                continue 
              else:
                continue

              for g in split6:
                split_name6 = g[0] 
                new_df6= g[1]
                new_dict6 = g[2]
                counter_lvl7 = 0
                stop_counter7 = 0
                depth_counter = 7

                min_att7 = info_min_attribute(new_dict6,new_df6,y_col)
                split7 = info_min_split(min_att7[0],new_dict6,new_df6,y_col)
                info_gain_6_7 = min_att5[1] - min_att6[1]

                if info_gain_6_7 >= min_info_gain and depth_counter<= max_depth:
                  tree_ary[counter_lvl0][1][counter_lvl1][1][counter_lvl2][1][counter_lvl3][1][counter_lvl4][1][counter_lvl5][1].append([split_name6, []])
            
        
                elif stop_counter6 == 0:
                  tree_ary[counter_lvl0][1][counter_lvl1][1][counter_lvl2][1][counter_lvl3][1][counter_lvl4][1][counter_lvl5][1].append([split_prediction(new_df5,y_col)])
                  stop_counter6+=1 
                  continue 
                else:
                  continue

                for h in split7:
                  split_name7 = h[0] 
                  new_df7= h[1]
                  new_dict7 = h[2]
                  counter_lvl8 = 0
                  stop_counter8 = 0
                  depth_counter = 8

                  min_att8 = info_min_attribute(new_dict7,new_df7,y_col)
                  split8 = info_min_split(min_att8[0],new_dict7,new_df7,y_col)
                  info_gain_7_8 = min_att6[1] - min_att7[1]

                  if info_gain_7_8 >= min_info_gain and depth_counter<= max_depth:
                    tree_ary[counter_lvl0][1][counter_lvl1][1][counter_lvl2][1][counter_lvl3][1][counter_lvl4][1][counter_lvl5][1][counter_lvl6][1].append([split_name7, []])
            
        
                  elif stop_counter7 == 0:
                    tree_ary[counter_lvl0][1][counter_lvl1][1][counter_lvl2][1][counter_lvl3][1][counter_lvl4][1][counter_lvl5][1][counter_lvl6][1].append([split_prediction(new_df6,y_col)])
                    stop_counter7+=1 
                    continue 
                  else:
                    continue

                  for i in split8:
                    split_name8 = i[0] 
                    new_df8= i[1]
                    new_dict8 = i[2]
                    counter_lvl9 = 0
                    stop_counter9 = 0
                    depth_counter = 9

                    min_att9 = info_min_attribute(new_dict8,new_df8,y_col)
                    split9 = info_min_split(min_att9[0],new_dict8,new_df8,y_col)
                    info_gain_8_9 = min_att7[1] - min_att8[1]

                    if info_gain_8_9 >= min_info_gain and depth_counter<= max_depth:
                      tree_ary[counter_lvl0][1][counter_lvl1][1][counter_lvl2][1][counter_lvl3][1][counter_lvl4][1][counter_lvl5][1][counter_lvl6][1][counter_lvl7][1].append([split_name8, []])
            
        
                    elif stop_counter8 == 0:
                      tree_ary[counter_lvl0][1][counter_lvl1][1][counter_lvl2][1][counter_lvl3][1][counter_lvl4][1][counter_lvl5][1][counter_lvl6][1][counter_lvl7][1].append([split_prediction(new_df7,y_col)])
                      stop_counter8+=1 
                      continue 
                    else:
                      continue


                    for j in split9:
                      split_name9 = j[0] 
                      new_df9= j[1]
                      new_dict9 = j[2]
                      counter_lvl10= 0
                      stop_counter10 = 0
                      depth_counter = 10

                      min_att10 = info_min_attribute(new_dict9,new_df9,y_col)
                      split10 = info_min_split(min_att10[0],new_dict9,new_df9,y_col)
                      info_gain_9_10 = min_att8[1] - min_att9[1]

                      if info_gain_9_10 >= min_info_gain and depth_counter<= max_depth:
                        tree_ary[counter_lvl0][1][counter_lvl1][1][counter_lvl2][1][counter_lvl3][1][counter_lvl4][1][counter_lvl5][1][counter_lvl6][1][counter_lvl7][1][counter_lvl8][1].append([split_name9, []])
            
        
                      elif stop_counter9 == 0:
                        tree_ary[counter_lvl0][1][counter_lvl1][1][counter_lvl2][1][counter_lvl3][1][counter_lvl4][1][counter_lvl5][1][counter_lvl6][1][counter_lvl7][1][counter_lvl8][1].append([split_prediction(new_df8,y_col)])
                        stop_counter9+=1 
                        continue 
                      else:
                        continue


                          
                                     
                        counter_lvl10+=1 
                      counter_lvl9+=1 
                    counter_lvl8+=1          
                  counter_lvl7+=1
                counter_lvl6+=1
              counter_lvl5+=1
            counter_lvl4+=1
          counter_lvl3+=1
        counter_lvl2+=1
      counter_lvl1+=1
    counter_lvl0+=1


  
  return tree_ary
 

In [ ]:
#tree_ary=train_tree(column_dict,df,0.01,8,"HeartDisease")


In [ ]:
#tree_ary[0]

# 6. Predict by using pretrained Tree

In [ ]:
def check_type(lvl_list): #check if the given list containts floats -->stop because end of tree reached
  converted_list = np.array(lvl_list)
  return converted_list.dtype == "float64"

In [ ]:
def class_prob(lvl_list, multi): #multily final node prediction with multiplied path
  converted_list = np.array(lvl_list)
  return converted_list * multi

In [ ]:
def sum_class_prob(predict_ary): #calculate final prediction for one patient by iteration through all path prediction arrays
  y_0 = 0
  y_1 = 0
  y_2 = 0
  class_prob = []
  for i in range(0, len(predict_ary)):
    try:
      y_0 += predict_ary[i][0]
    except:
      y_0 += 0
    try:
      y_1 += predict_ary[i][1]
    except:
      y_1 += 0
    try:
      y_2 += predict_ary[i][2]
    except:
      y_2 += 0
  class_prob.append([y_0,y_1,y_2])
  return class_prob

In [ ]:
def predict(trained_tree, df): #predict Diabetes target value distribution for every instance by given dataframe and trained tree
  
  predictproba_diabetes_all_patients = []
  for index, row in df.iterrows(): #get every instance step by step from the given test data and predict for it the probabilities 
    predict_ary = []
    for split1 in range(0,len(trained_tree)):
      lvl1=trained_tree[split1][0]
      lvl1_value = row[lvl1]

      for split2 in range(0,len(trained_tree[split1][1])):
        lvl2=trained_tree[split1][1][split2][0] #HighBP =0
        
        if check_type(lvl2) == True:
          predict_ary.append(class_prob(lvl2,lvl1_value))
          continue
        else:
          lvl2_value = row[lvl2]
          multi1_2 = lvl1_value * lvl2_value

          for split3 in range(0,len(trained_tree[split1][1][split2][1])):
              lvl3=trained_tree[split1][1][split2][1][split3][0] #Age=4

              if check_type(lvl3) == True:
                predict_ary.append(class_prob(lvl3,multi1_2))

                continue
              else:
                lvl3_value = row[lvl3]
                multi2_3 = lvl3_value * multi1_2

                for split4 in range(0,len(trained_tree[split1][1][split2][1][split3][1])):
                  lvl4=trained_tree[split1][1][split2][1][split3][1][split4][0]  
                  if check_type(lvl4) == True:
                    predict_ary.append(class_prob(lvl4,multi2_3))

                    continue
                  else:
                    lvl4_value = row[lvl4]
                    multi3_4 = lvl4_value * multi2_3
                  
                    for split5 in range(0,len(trained_tree[split1][1][split2][1][split3][1][split4][1])):
                      lvl5=trained_tree[split1][1][split2][1][split3][1][split4][1][split5][0]  
                      if check_type(lvl5) == True:
                        predict_ary.append(class_prob(lvl5,multi3_4))

                        continue
                      else:
                        lvl5_value = row[lvl5]
                        multi4_5 = lvl5_value * multi3_4
                      
                        for split6 in range(0,len(trained_tree[split1][1][split2][1][split3][1][split4][1][split5][1])):
                          lvl6=trained_tree[split1][1][split2][1][split3][1][split4][1][split5][1][split6][0]  
                          if check_type(lvl6) == True:
                            predict_ary.append(class_prob(lvl6,multi4_5))

                            continue
                          else:
                            lvl6_value = row[lvl6]
                            multi5_6 = lvl6_value * multi4_5

                            for split7 in range(0,len(trained_tree[split1][1][split2][1][split3][1][split4][1][split5][1][split6][1])):
                              lvl7=trained_tree[split1][1][split2][1][split3][1][split4][1][split5][1][split6][1][split7][0]   
                              if check_type(lvl7) == True:
                                predict_ary.append(class_prob(lvl7,multi5_6))

                                continue
                              else:
                                lvl7_value = row[lvl7]
                                multi6_7 = lvl7_value * multi5_6

                                for split8 in range(0,len(trained_tree[split1][1][split2][1][split3][1][split4][1][split5][1][split6][1][split7][1])):
                                  lvl8=trained_tree[split1][1][split2][1][split3][1][split4][1][split5][1][split6][1][split7][1][split8][0]    
                                  if check_type(lvl8) == True:
                                    predict_ary.append(class_prob(lvl8,multi6_7))

                                    continue
                                  else:
                                    lvl8_value = row[lvl8]
                                    multi7_8 = lvl8_value * multi6_7 

                                    for split9 in range(0,len(trained_tree[split1][1][split2][1][split3][1][split4][1][split5][1][split6][1][split7][1][split8][1])):
                                      lvl9=trained_tree[split1][1][split2][1][split3][1][split4][1][split5][1][split6][1][split7][1][split8][1][split9][0]   
                                      if check_type(lvl9) == True:
                                        predict_ary.append(class_prob(lvl9,multi7_8))

                                        continue
                                      else:
                                        lvl9_value = row[lvl9]
                                        multi8_9 = lvl9_value * multi7_8 

                                        for split10 in range(0,len(trained_tree[split1][1][split2][1][split3][1][split4][1][split5][1][split6][1][split7][1][split8][1][split9][1])):
                                          lvl10=trained_tree[split1][1][split2][1][split3][1][split4][1][split5][1][split6][1][split7][1][split8][1][split9][1][split10][0]   
                                          if check_type(lvl10) == True:
                                            predict_ary.append(class_prob(lvl10,multi8_9))

                                            continue
                                          else:
                                            lvl10_value = row[lvl10]
                                            multi9_10 = lvl10_value * multi8_9 


    predictproba_diabetes_all_patients.append(sum_class_prob(predict_ary))
  return predictproba_diabetes_all_patients 


In [ ]:
#b = predict(tree_ary,df[:600])
#b

# 7. Probabilistic Random Forest Algorithm

In [ ]:
def bootstrapping(c_dict,df,validation_size,n_features): #give column dictonary and df as well as the number of instances and features after bagging
  df_bt_train = df.copy() #bt: Bootstrapped for training the trees
  c_dict_bt = c_dict.copy()

  df_bt_train = df_bt_train[0:0]
  df_bt_validation = df_bt_train.copy() #bt: All rows which are not in train sample are in validation sample for MSE Calucaltion of the tree
  
  n_instances = int(round(len(df) * (1 -  validation_size),0))
  df_bt_train = df.sample(n = n_instances, replace = True) #pick randomly given number of instances from dataframe with replacement

  random_features = []
  for key, value in sorted(c_dict.items(), key=lambda x: random.random()): #randomly pick keys(columns/features) from column_dict
    random_features.append(key)
  
  random_features = random_features[0:(len(random_features)-n_features)]
  #print(random_features) see the selected number of features which will be deleted ['PhysHlth', 'HighChol', 'AnyHealthcare',
  delete_occurrences = []
  for occurrences in random_features:
    delete_occurrences.extend(c_dict[occurrences])

  #print(delete_occurrences) see the occurrences coresponding to the features ['PhysHlth_cat=1', 'PhysHlth_cat=0', 'HighChol=1', 'HighChol=0',...

  for occurrences in delete_occurrences:  #delete the chosen occurrences from the dataframe
    df_bt_train.drop(columns=occurrences, axis=1,inplace = True)
  
  for feature in random_features:
    c_dict_bt.pop(feature)

  df_bt_validation = df.drop(df_bt_train.index) #get all the rows which are not in the bootstrapped train dataframe

  return df_bt_train,df_bt_validation, c_dict_bt

In [ ]:
def y_encoding(df,y_col): #one hot encoding of true y of every instance
  y_total_df = []
  for index, row in df.iterrows():
    y_instance = [0,0,0]
    if row[y_col] == 0:
      y_instance[0] = 1
    elif row[y_col] == 1:
      y_instance[1] = 1
    elif row[y_col] == 2:
      y_instance[2] = 1
    y_total_df.append(y_instance)
  return y_total_df #return the hole df the one hot encoding in one array

In [ ]:
def mean_squared_error_tree(df,y_encoding,prediction): #calcualte mean squared error for prediction array of tree and the one hot encoding array of real value y
  mse_total = 0
  for i in range(0,len(df)):

    mse_total += (y_encoding[i][0] - prediction[i][0][0])**2 + (y_encoding[i][1] - prediction[i][0][1])**2 + (y_encoding[i][2] - prediction[i][0][2])**2
 
  return (mse_total / len(df))   

In [ ]:
def random_forest_fit(c_dict,df_train,y_col, n_trees,n_trees_final, min_info_gain,max_depth,validation_size, n_features,methode):

  my_dict = {}                       #safes every tree as key with its coresponding value as value
  my_dict_key = {}  
  n_trees_final_ary = []             #safe n (n_trees_final) number of best trees as trained trees and return it
  counter = 0
  for tree in tqdm(range(0,n_trees)): #create n_trees number of trees 
    #for every tree do a bootstrapping of rows and columns, split given train dataframe (df) into train and validation sample
    df_bt_train,df_bt_validation, c_dict_bt = bootstrapping(c_dict,df_train,validation_size,n_features) 
    y_encode_train = y_encoding(df_bt_train,y_col)
    y_encode_validation = y_encoding(df_bt_validation,y_col)
    #train every tree based on the bootstrapping
    trained_tree = train_tree(c_dict_bt, df_bt_train, min_info_gain,max_depth,y_col)
    #make a prediction for every instance based on the trained tree
    prediction_train = predict(trained_tree, df_bt_train)
    prediction_test = predict(trained_tree, df_bt_validation)

    if methode == "mse":
     #calculate mean squared error for the given tree for train and test data
     train_mse = mean_squared_error_tree(df_bt_train, y_encode_train, prediction_train)
     validation_mse = mean_squared_error_tree(df_bt_validation, y_encode_validation, prediction_test) #get mse für validation data sete

     my_dict_key.update({counter: trained_tree})
     my_dict.update({counter: validation_mse})
     my_dict_key.update({counter: trained_tree})
     my_dict.update({counter: validation_mse})
     counter+=1

    elif methode == "accuracy":
      #calculate accuracy and take it choosing the trees
      y_true_val = df_bt_validation["HeartDisease"]
      y_pred_val = convert_prob_to_class(prediction_test)
      y_true_train = df_bt_train["HeartDisease"]
      y_pred_train = convert_prob_to_class(prediction_train)
      accuracy_val = accuracy_score(y_true_val, y_pred_val)
      accuracy_train = accuracy_score(y_true_train, y_pred_train)
      print("Train Accuracy: ",accuracy_train)
      print("Validation Accuracy: ",accuracy_val) 
      my_dict_key.update({counter: trained_tree})
      my_dict.update({counter: accuracy_val})
      counter+=1
      


    
  if methode == "mse":
   # Finding n = n_trees_final number of smalest mse
   get_counter_keys = nsmallest(n_trees_final, my_dict, key = my_dict.get)
   for i in get_counter_keys:
     n_trees_final_ary.append(my_dict_key[i])
  elif methode == "accuracy":
   get_counter_keys = sorted(range(len(my_dict)), key = lambda sub: my_dict[sub])[-n_trees_final:]
   for i in get_counter_keys:
     n_trees_final_ary.append(my_dict_key[i])



  return n_trees_final_ary

In [ ]:
#n_trees_final_ary = random_forest_fit(c_dict= column_dict,df_train= df,y_col ="HeartDisease",n_trees= 10,n_trees_final= 3, min_info_gain= 0.01,max_depth=6,validation_size = 0.25, n_features=15,methode="accuracy")

In [ ]:
def random_forest_classifier(n_trees_final_ary, df_test, c_dict):    #predicts classes/y based on the best trees from the Random Forrest
 safe_prediction = []
 final_prediction = []
 for tree in n_trees_final_ary:
   prediction_test = predict(tree, df_test)
   safe_prediction.append(prediction_test)
 

 for y_pred_j in range(0,len(df_test)):
    y_0 = 0
    y_1 = 0
    y_2 = 0

    for y_pred_i in range(0,len(safe_prediction)): #iterate for one instance trough every prediction from the given trees and take the mean
      y_0 += safe_prediction[ y_pred_i][y_pred_j][0][0]
      y_1 += safe_prediction[ y_pred_i][y_pred_j][0][1]
      y_2 += safe_prediction[ y_pred_i][y_pred_j][0][2] 

    y_0_mean = y_0/len(safe_prediction)
    y_1_mean = y_1/len(safe_prediction)
    y_2_mean = y_2/len(safe_prediction)
    final_prediction.append([y_0_mean, y_1_mean, y_2_mean])

   
 return final_prediction



In [ ]:
#final_prediction = random_forest_classifier(n_trees_final_ary,df[:10],column_dict)

In [ ]:
#final_prediction

# Example Evaluation

In [ ]:
def convert_prob_to_class(final_prediction): 
 #converte the probability array final prediction into categorial data based on highest probability
  final_prediction_class = []
  for i in final_prediction:
    i = np.array(i)
    maxindex = i.argmax()
    final_prediction_class.append(maxindex)

  return final_prediction_class

In [ ]:

dftrain = df_splitknn[:600]
dftest = df_splitknn[600:]

#fit the random forest
n_trees_final_ary = random_forest_fit(c_dict= column_dict, df_train = dftrain, y_col ="HeartDisease",n_trees = 5,
                                      n_trees_final = 3, min_info_gain = 0.1,max_depth = 4,validation_size = 0.2, n_features = 14, methode = "mse")

#predict with the random forest based on the test data set
final_prediction = random_forest_classifier(n_trees_final_ary, dftest, column_dict)


100%|██████████| 5/5 [03:58<00:00, 47.77s/it]


In [ ]:
dftest

,HeartDisease,Smoking_No,Smoking_Yes,AlcoholDrinking_No,AlcoholDrinking_Yes,DiffWalking_No,DiffWalking_Yes,Sex_Female,Sex_Male,Race_American Indian/Alaskan Native,...,AgeCategory_cat_1,AgeCategory_cat_2,AgeCategory_cat_3,AgeCategory_cat_4,AgeCategory_cat_5,SleepTime_cat_0,SleepTime_cat_1,SleepTime_cat_2,SleepTime_cat_3,SleepTime_cat_4
600,NaN,1.000,0.000,0.818182,0.181818,1.000,0.000,0.0,1.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,0.00,0.0,0.0
601,NaN,0.625,0.375,1.000000,0.000000,0.875,0.125,1.0,0.0,0.0,...,0.000000,0.111111,0.444444,0.000000,0.0,0.000000,0.300000,0.70,0.0,0.0
602,NaN,1.000,0.000,0.800000,0.200000,1.000,0.000,0.0,1.0,0.0,...,0.111111,0.333333,0.000000,0.111111,0.0,0.083333,0.666667,0.25,0.0,0.0


In [ ]:
print("Thomas hat mit einer Wahrscheinlichkeit von ",(final_prediction[0][0]*100),"% kein Herzerkrankung")
print("Alina hat mit einer Wahrscheinlichkeit von ",(final_prediction[1][0]*100),"% kein Herzerkrankung")
print("Lucas hat mit einer Wahrscheinlichkeit von ",(final_prediction[2][0]*100),"% kein Herzerkrankung")

Thomas hat mit einer Wahrscheinlichkeit von  79.30615589690667 % kein Herzerkrankung
Alina hat mit einer Wahrscheinlichkeit von  66.69711749395088 % kein Herzerkrankung
Lucas hat mit einer Wahrscheinlichkeit von  63.4605051733621 % kein Herzerkrankung
